In [0]:
print("Welcome Hunter")

Welcome Hunter


In [0]:
spark.conf.set(
  "fs.azure.account.key.Your Storage Account.dfs.core.windows.net",
  "Your Access Key"
)


In [0]:
df_store = spark.read.parquet("abfss://retailsdb@hellooodb.dfs.core.windows.net/Raw_Data/dbo.stores.parquet")
df_store.show(5)

+--------+--------------------+------------+
|store_id|          store_name|    location|
+--------+--------------------+------------+
|       1|     City Mall Store|         UAE|
|       2|   High Street Store|Saudi Arabia|
|       3|   Tech World Outlet|       Qatar|
|       4|Cairo Festival Ci...|       Egypt|
|       5|          Mega Plaza|      Kuwait|
+--------+--------------------+------------+



In [0]:
df_transaction = spark.read.parquet("abfss://retailsdb@hellooodb.dfs.core.windows.net/Raw_Data/dbo.transactions.parquet")
df_transaction.show(5)

+--------------+-----------+----------+--------+--------+----------------+
|transaction_id|customer_id|product_id|store_id|quantity|transaction_date|
+--------------+-----------+----------+--------+--------+----------------+
|            31|        101|         1|       1|       3|      2025-04-01|
|            32|        102|         2|       2|       2|      2025-04-03|
|            33|        103|         3|       3|       1|      2025-04-05|
|            34|        104|         4|       4|       5|      2025-04-07|
|            35|        105|         5|       5|       2|      2025-04-09|
+--------------+-----------+----------+--------+--------+----------------+
only showing top 5 rows


In [0]:
df_products = spark.read.parquet("abfss://retailsdb@hellooodb.dfs.core.windows.net/Raw_Data/dbo.products.parquet")
df_products.show(5)

+----------+-----------------+-----------+-----+
|product_id|     product_name|   category|price|
+----------+-----------------+-----------+-----+
|         1|   Wireless Mouse|Electronics|  800|
|         2|Bluetooth Speaker|Electronics| 1200|
|         3|         Yoga Mat|    Fitness|  499|
|         4|     Laptop Stand|Accessories|  999|
|         5|     Notebook Set| Stationery|  149|
+----------+-----------------+-----------+-----+
only showing top 5 rows


In [0]:
df_Customer = spark.read.parquet("abfss://retailsdb@hellooodb.dfs.core.windows.net/Raw_Data/MohammedHameds/Retails_Project/refs/heads/main/dataset/customers.parquet")
df_Customer.show(5)

+-----------+----------------+--------------------+------------+-----------------+
|customer_id|       full_name|               email|     country|registration_date|
+-----------+----------------+--------------------+------------+-----------------+
|        101|    Ahmed Khaled|ahmed.khaled1@gma...|       Egypt|       2025-10-16|
|        102| Sara Al Mansour|sara.almansour2@o...|Saudi Arabia|       2025-10-18|
|        103|    Layla Kazemi|layla.kazemi3@yah...|         UAE|       2025-10-19|
|        104|     Omar Farouk|omar.farouk4@gmai...|       Egypt|       2025-10-17|
|        105|Fatima Al Rashid|fatima.alrashid5@...|Saudi Arabia|       2025-10-15|
+-----------+----------------+--------------------+------------+-----------------+
only showing top 5 rows


In [0]:
df_store.write.mode("overwrite").saveAsTable("polin.sales.store_bronze")
df_transaction.write.mode("overwrite").saveAsTable("polin.sales.transaction_bronze")
df_products.write.mode("overwrite").saveAsTable("polin.sales.product_bronze")
df_Customer.write.mode("overwrite").saveAsTable("polin.sales.customer_bronze")

In [0]:
df_Customer.printSchema()

root
 |-- customer_id: long (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: string (nullable = true)



In [0]:
from pyspark.sql.functions import col, sum , count, countDistinct, first

In [0]:
df_Customer = df_Customer.select(
    col("customer_id").cast("int"),
    col("full_name"),
    col("email"),
    col("country"),
    col("registration_date").cast("date")
)
df_Customer.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- full_name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- registration_date: date (nullable = true)



In [0]:
df_products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- category: string (nullable = true)
 |-- price: integer (nullable = true)



In [0]:
df_transaction.printSchema()

root
 |-- transaction_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- transaction_date: date (nullable = true)



In [0]:
df_retails_silver = df_transaction.join(df_Customer,"customer_id").join(df_products,"product_id").join(df_store, "store_id").withColumn("total_amount", col("quantity") * col("price"))
display(df_retails_silver)

store_id,product_id,customer_id,transaction_id,quantity,transaction_date,full_name,email,country,registration_date,product_name,category,price,store_name,location,total_amount
3,8,101,58,3,2025-05-25,Ahmed Khaled,ahmed.khaled1@gmail.com,Egypt,2025-10-16,Desk Organizer,Accessories,399,Tech World Outlet,Qatar,1197
4,9,102,59,5,2025-05-27,Sara Al Mansour,sara.almansour2@outlook.com,Saudi Arabia,2025-10-18,Dumbbell Set,Fitness,1999,Cairo Festival City Mall,Egypt,9995
5,10,103,60,1,2025-05-29,Layla Kazemi,layla.kazemi3@yahoo.com,UAE,2025-10-19,Pen Drive 32GB,Electronics,599,Mega Plaza,Kuwait,599
1,1,104,61,2,2025-06-01,Omar Farouk,omar.farouk4@gmail.com,Egypt,2025-10-17,Wireless Mouse,Electronics,800,City Mall Store,UAE,1600
2,2,105,62,3,2025-06-03,Fatima Al Rashid,fatima.alrashid5@hotmail.com,Saudi Arabia,2025-10-15,Bluetooth Speaker,Electronics,1200,High Street Store,Saudi Arabia,3600
3,3,106,63,4,2025-06-05,Yousef Nasser,yousef.nasser6@gmail.com,UAE,2025-10-16,Yoga Mat,Fitness,499,Tech World Outlet,Qatar,1996
4,4,107,64,5,2025-06-07,Mona Hossam,mona.hossam7@yahoo.com,Egypt,2025-10-18,Laptop Stand,Accessories,999,Cairo Festival City Mall,Egypt,4995
5,5,108,65,1,2025-06-09,Hassan Al Saud,hassan.alsaud8@outlook.com,Saudi Arabia,2025-10-19,Notebook Set,Stationery,149,Mega Plaza,Kuwait,149
1,6,109,66,2,2025-06-11,Ayesha Khan,ayesha.khan9@gmail.com,UAE,2025-10-15,Water Bottle,Fitness,299,City Mall Store,UAE,598
2,7,110,67,3,2025-06-13,Tamer Adel,tamer.adel10@hotmail.com,Egypt,2025-10-17,Smartwatch,Electronics,4999,High Street Store,Saudi Arabia,14997


In [0]:
df_retails_silver.write.mode("overwrite").saveAsTable("polin.sales.df_retails_silver")

In [0]:
df_country_customer = df_retails_silver.groupBy("country").agg(count("customer_id").alias("Count_Of_Customrs"))
display(df_country_customer)

country,Count_Of_Customrs
Saudi Arabia,17
UAE,16
Egypt,17


In [0]:
df_country_customer.write.mode("overwrite").saveAsTable("polin.sales.country_customer_gold")

In [0]:
df_Contry_transaction = df_retails_silver.groupBy("country", "category").agg(count("transaction_id").alias("Count_transaction"))
display(df_Contry_transaction)

country,category,Count_transaction
Saudi Arabia,Fitness,3
UAE,Stationery,2
UAE,Electronics,5
Egypt,Electronics,7
UAE,Fitness,6
Egypt,Accessories,3
Saudi Arabia,Accessories,4
Egypt,Fitness,6
Egypt,Stationery,1
Saudi Arabia,Electronics,8


In [0]:
df_Contry_transaction.write.mode("overwrite").saveAsTable("polin.sales.Contry_transaction_gold")

In [0]:
df_top_products = df_retails_silver.groupby(["country", "product_name"]).agg(sum("total_amount").alias("total_amount"))
df_top_products = df_top_products.orderBy(["country", "total_amount"], ascending=[True, False])
display(df_top_products)


country,product_name,total_amount
Egypt,Smartwatch,19996
Egypt,Dumbbell Set,9995
Egypt,Laptop Stand,9990
Egypt,Bluetooth Speaker,4800
Egypt,Wireless Mouse,4000
Egypt,Pen Drive 32GB,3594
Egypt,Yoga Mat,1497
Egypt,Desk Organizer,1197
Egypt,Water Bottle,1196
Egypt,Notebook Set,149


In [0]:
df_top_products.write.mode("overwrite").saveAsTable("polin.sales.top_products_gold")

In [0]:
df_loyalty = df_retails_silver.groupBy("customer_id").agg(countDistinct("transaction_id").alias("transactions_per_customer"),first("full_name").alias("full_name"))
display(df_loyalty)

customer_id,transactions_per_customer,full_name
101,2,Ahmed Khaled
102,2,Sara Al Mansour
103,2,Layla Kazemi
104,2,Omar Farouk
105,2,Fatima Al Rashid
106,2,Yousef Nasser
107,2,Mona Hossam
108,2,Hassan Al Saud
109,2,Ayesha Khan
110,2,Tamer Adel


In [0]:
df_loyalty.write.mode("overwrite").saveAsTable("polin.sales.loyalty_gold")

In [0]:
df_category_sales = df_retails_silver.groupby("category").agg(sum("total_amount").alias("total_amount")).sort("total_amount", ascending=False)
display(df_category_sales)

category,total_amount
Electronics,104175
Fitness,43159
Accessories,28761
Stationery,1490


In [0]:
df_category_sales.write.mode("overwrite").saveAsTable("polin.sales.category_sales_gold")

In [0]:
df_country_revenue = df_retails_silver.groupby("country").agg(sum("total_amount").alias("total_amount")).sort("total_amount", ascending=False)
display(df_country_revenue)

country,total_amount
Saudi Arabia,74010
Egypt,56414
UAE,47161


In [0]:
df_country_revenue.write.mode("overwrite").saveAsTable("polin.sales.country_revenue_gold")

In [0]:
df_store_sales = df_retails_silver.groupby("store_name").agg(sum("total_amount").alias("total_amount"))
display(df_store_sales)

store_name,total_amount
High Street Store,87986
Cairo Festival City Mall,55961
Mega Plaza,8079
Tech World Outlet,11773
City Mall Store,13786


In [0]:
df_store_sales.write.mode("overwrite").saveAsTable("polin.sales.store_sales_gold")

In [0]:
df_price_sensitivity = df_retails_silver.groupby("product_name").agg({"price": "mean", "quantity": "mean"}).sort("mean(quantity)", ascending=False)
display(df_price_sensitivity)

product_name,mean(price),mean(quantity)
Laptop Stand,999.0,4.4
Desk Organizer,399.0,3.4
Dumbbell Set,1999.0,3.4
Bluetooth Speaker,1200.0,3.0
Smartwatch,4999.0,2.8
Water Bottle,299.0,2.8
Wireless Mouse,800.0,2.4
Pen Drive 32GB,599.0,2.2
Yoga Mat,499.0,2.0
Notebook Set,149.0,2.0


In [0]:
df_price_sensitivity = df_price_sensitivity.withColumnRenamed("mean(price)", "mean_price").withColumnRenamed("mean(quantity)", "mean_quantity")

In [0]:
display(df_price_sensitivity)

product_name,mean_price,mean_quantity
Laptop Stand,999.0,4.4
Desk Organizer,399.0,3.4
Dumbbell Set,1999.0,3.4
Bluetooth Speaker,1200.0,3.0
Smartwatch,4999.0,2.8
Water Bottle,299.0,2.8
Wireless Mouse,800.0,2.4
Pen Drive 32GB,599.0,2.2
Yoga Mat,499.0,2.0
Notebook Set,149.0,2.0


In [0]:
df_price_sensitivity.write.mode("overwrite").saveAsTable("polin.sales.price_sensitivity_gold")